In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM, AutoTokenizer

/home/cosmoquester/aibookathon/env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
summarize_model_name: str = "digit82/kobart-summarization"
summarize_num_beams: int = 4
sum_input_max_length: int = 512
sum_max_token_length: int = 128

model_name: str = "../model-00epoch-7000steps-4.3906ppl"
tokenizer_name: str = model_name
prompt_max_length: int = 128
max_length: int = 512
device: str = "cuda"
auth_token: str = None

num_generate_characters: int = 20500
use_n_recent_summarizations: int = 10

In [3]:
tokenizer_kwargs = {"padding_side": "left", "truncation_side": "left"}
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_auth_token=auth_token, **tokenizer_kwargs)
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=auth_token).to(device)

In [4]:
tokenizer_kwargs = {"padding_side": "left", "truncation_side": "left"}
summarize_tokenizer = AutoTokenizer.from_pretrained(summarize_model_name, use_auth_token=auth_token, **tokenizer_kwargs)
summarize_model = AutoModelForSeq2SeqLM.from_pretrained(summarize_model_name).to(device)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [5]:
def datum_to_string(datum, use_n_summary: int = 10):
    text = f"제목: {datum['title']}\n"

    summarizations = datum.get("summarizations")
    if summarizations:
        summarizations = summarizations[-use_n_summary:]
        summarization = " ".join(summarizations).replace("\n", " ")
        text = f"요약: {summarization}\n" + text
    return text

def segment_by_anychar(text: str, delimiters: str):
    segments = []
    last_index = 0
    for i, char in enumerate(text):
        if char in delimiters:
            segments.append(text[last_index: i + 1])
            last_index = i + 1
    return segments

In [6]:
initial_input_example = {
    "title": "나의 빛과 어둠", 
    "summarizations": [], 
    "original_texts": [],
    "contents": []
}
seed: int = 42
backup_indices = [] # Used for replay already found indices or continue writing from special part

In [7]:
from IPython.display import clear_output
from copy import deepcopy
from proto.request import request_log

torch.manual_seed(seed)
input_example = deepcopy(initial_input_example)
select_indices = []

while len("".join(input_example["contents"])) < num_generate_characters:
  prompt = datum_to_string(input_example)
  input_ids = tokenizer(
          [prompt],
          add_special_tokens=False,
          max_length=prompt_max_length,
          padding="longest",
          truncation=True,
          return_tensors="pt",
          return_token_type_ids=False,
          return_attention_mask=False,
  )["input_ids"].to(device)

  output = model.generate(input_ids, 
      max_length, 
      do_sample=True,
      num_return_sequences=1,
      # repetition_penalty=1.2,
      # num_beams=5,
      # temperature=2.0,
      # top_p=0.99,
      # top_k=3,
      pad_token_id=tokenizer.pad_token_id,
      use_cache=True,
  )
  output = output.squeeze(dim=0)[input_ids.size(1):]

  text = tokenizer.decode(output, skip_special_tokens=True)
  segments = segment_by_anychar(text, ".?!\n'\"“”‘’")
  segments.insert(0, "")

  print("[TITLE]")
  print(input_example["title"])
  print("\n[SUMMARIZATIONS]")
  for i, summarization in enumerate(input_example["summarizations"], start=ord("A")):
    print(f"[{chr(i)}] {summarization}")
  print("\n[KEEP CONTENT]")
  print("".join(input_example["contents"]))
  print("\n[CUR SENTENCES]")
  for i, segment in enumerate(segments):
    print(f"[{i}] {segment}")
  print("[-1] ※ EXIT: 더 이상 문장을 생성하지 않습니다.")
  print(flush=True, end="")

  if backup_indices:
    select_index = backup_indices.pop(0)
  else:
    while True:
      try:
        select_index = int(input("Select last sentence index: "))
        break
      except:
        continue

  select_indices.append(select_index)
  if select_index < 0:
    break

  request_log(prompt=prompt, generated=text)
  clear_output()

  print("Selected Indices:", select_indices, flush=True)
  
  selected_segment = segments[:select_index + 1]
  selected_text = "".join(selected_segment)

  if not selected_text:
    continue

  input_example["contents"].append(selected_text)
  input_example["original_texts"].append(text)
  sum_input_ids = summarize_tokenizer(
          [selected_text],
          add_special_tokens=False,
          max_length=sum_input_max_length,
          padding="longest",
          truncation=True,
          return_tensors="pt",
          return_token_type_ids=False,
          return_attention_mask=False,
  )["input_ids"].to(device)

  sum_outputs = summarize_model.generate(sum_input_ids, 
      sum_max_token_length, 
      num_beams=summarize_num_beams,
      num_return_sequences=1,
      eos_token_id=summarize_tokenizer.eos_token_id,
      pad_token_id=summarize_tokenizer.pad_token_id,
      use_cache=True,
  )

  summarization = summarize_tokenizer.decode(sum_outputs.squeeze(dim=0), skip_special_tokens=True)
  input_example["summarizations"].append(summarization)
  input_example["summarizations"] = input_example["summarizations"][-use_n_recent_summarizations:]

Selected Indices: [14, 8]
[TITLE]
나의 빛과 어둠

[SUMMARIZATIONS]
[A] 나의 빛과 어둠 나라는 나의 빛과 어둠이 공동체학교에 입학한 지 한 달쯤 되었을 때였는데, 학교 가는 길에 하늘을 본 적이 있냐고 누군가 내게 묻는다면 나는 늘 망설임 없이 하늘을 본 적이 있다고 대답했다.
[B] 처음에는 하늘을 본다는 것이 무슨 의미일까 잘 몰랐고 그 의미에 대해 곰곰이 생각해 본 적도 없었지만 하늘 위에는 별들이 환하게 빛나고 있었고 나는 하늘을 보는 것이 처음이라 저 멀리서 보이는 별이 까만 어둠 속에서 빛나는 빛을 보고 있다는 사실이 너무 크게 가슴 벅찼으며 삶이 하늘을 보고 살아야 하는 것은 아닐까, 어떤 순간에도 삶은 그 순간만을 위해 존재하는 것일까라고 생각했다.

[KEEP CONTENT]
나의 빛과 어둠.
내 나이 열한 살인 어느 날, 나에겐 빛이 있었고 어둠이 있었다.
학교에 입학한 지 한 달쯤 되었을 때였나 보다.
평소와 다름없이 아침에 일어나면 엄마의 출근 준비를 도왔고 씻고 준비를 마친 나는 여느 날처럼 학교로 향했다. 학교는 가던 길 그대로 쭉 따라 올라갔다. 학교 가는 길에 하늘을 본 적이 있냐고 누군가 내게 묻는다면, 나는 늘 망설임 없이 하늘을 본다. 비가 오는지, 구름이 가득 껴서 해가 보이질 않는지, 하늘을 올려다보며 오늘따라 왜 더 선명하게 보이는 것만 같은지 의문을 품기도 했었다.
그런데 그날은 달랐다. 유독 구름이 없었다 내가 걷던 이 길이 먹구름의 바다가 되어 내 방 호수까지 닿을 듯한 그런 하늘을 보고 있는데 갑자기 엄마가 집에 간다고 내게 손을 흔들며 뒤돌아보지 말라고 했다. 나는 하늘에 있는 구름이 너무 예뻐서 그런가 보다 했는데 엄마가 더 이상 내 눈앞에 없어서 하늘을 올려다보니, 하늘의 구름도 한 점 보이지 않았다.처음에는 하늘을 본다는 것이 무슨 의미일까 잘 몰랐으며, 그 의미에 대해 곰곰이 생각해 본 적도 없었다. 그러다가 나는 하늘을 보면서 사람이 하늘을 보고 있다는 사실이

In [8]:
print("Initial Input Example:", initial_input_example)
print("Seed:", seed)
print("Selected Indices:", select_indices)
print("※ 위 정보를 이용하면 똑같은 결과를 다시 재현할 수 있습니다. (selected indices를 backup indices로 사용)")

Initial Input Example: {'title': '나의 빛과 어둠', 'summarizations': [], 'original_texts': [], 'contents': []}
Seed: 42
Selected Indices: [14, 8, -1]
※ 위 정보를 이용하면 똑같은 결과를 다시 재현할 수 있습니다. (selected indices를 backup indices로 사용)


In [9]:
print("※ 혹시나 중간과정을 스킵하고 이어서 추론을 원한다면 현재 `input_example`을 복원하고 seed를 아래의 값으로 설정하세요.")
print("Current Seed:", torch.seed())

※ 혹시나 중간과정을 스킵하고 이어서 추론을 원한다면 현재 `input_example`을 복원하고 seed를 아래의 값으로 설정하세요.
Current Seed: 11684356136217954473


In [11]:
import json
import os

output_path = "essay-output.json"

if os.path.exists(output_path):
    raise ValueError("Already existing output path!")

with open(output_path, "w") as f:
    json.dump({
        "config": {
            "summarize_model_name": summarize_model_name,
            "summarize_num_beams": summarize_num_beams,
            "sum_input_max_length": sum_input_max_length,
            "sum_max_token_length": sum_max_token_length,
            "model_name": model_name,
            "tokenizer_name": tokenizer_name,
            "prompt_max_length": prompt_max_length,
            "max_length": max_length,
            "device": device,
            "auth_token": auth_token,
            "num_generate_characters": num_generate_characters,
            "use_n_recent_summarizations": use_n_recent_summarizations,
        },
        "initial": {
            "example": initial_input_example,
            "seed": seed,
        },
        "output": {
            "selected_indices": select_indices,
            "example": input_example,
        }
    }, f, ensure_ascii=False, indent=2)